In [3]:
import os
vctk = '/data1/DB/VCTK/VCTK/data/VCTK/wav48_silence_trimmed'
spks = os.listdir(vctk)
spks_test = ['p270', 'p256', 'p363', 'p241', 'p300', 'p336', 'p253', 'p266']
spks_train = [spk for spk in spks if spk not in spks_test]

npz_train = []
npz_test = []

In [4]:
spks_train

['p233',
 'p274',
 'p234',
 'p260',
 'log.txt',
 'p278',
 'p269',
 'p257',
 'p287',
 'p297',
 'p238',
 'p276',
 'p361',
 'p227',
 'p314',
 'p307',
 'p239',
 'p263',
 'p326',
 'p230',
 'p295',
 'p251',
 's5',
 'p340',
 'p313',
 'p226',
 'p311',
 'p286',
 'p318',
 'p312',
 'p335',
 'p281',
 'p376',
 'p229',
 'p255',
 'p282',
 'p294',
 'p240',
 'p305',
 'p310',
 'p283',
 'p306',
 'p259',
 'p316',
 'p279',
 'p245',
 'p252',
 'p250',
 'p275',
 'p285',
 'p292',
 'p243',
 'p343',
 'p330',
 'p237',
 'p288',
 'p254',
 'p315',
 'p374',
 'p236',
 'p268',
 'p317',
 'p262',
 'p267',
 'p225',
 'p329',
 'p362',
 'p264',
 'p244',
 'p261',
 'p228',
 'p302',
 'p247',
 'p231',
 'p308',
 'p303',
 'p272',
 'p280',
 'p333',
 'p347',
 'p232',
 'p299',
 'p277',
 'p246',
 'p364',
 'p258',
 'p345',
 'p341',
 'p284',
 'p304',
 'p249',
 'p248',
 'p323',
 'p271',
 'p273',
 'p298',
 'p265',
 'p293',
 'p339',
 'p301',
 'p334',
 'p360',
 'p351']

In [7]:
import os
import glob
from tqdm import tqdm
from multiprocessing import Pool
from functools import partial
import numpy as np

def mp_npz(npz):
    length = np.load(npz)['mel'].shape[1]
    if length >= 32 and length <= 1000:
        return [npz, length]

def make_filelist_vctk_train(input_dir, output_dir):
    spks = os.listdir(input_dir)
    spks_test = ['p270', 'p256', 'p363', 'p241', 'p300', 'p336', 'p253', 'p266']
    spks_train = [spk for spk in spks if spk not in spks_test]

    npz_train = []

    for spk in spks_train:
        npz_train += glob.glob(os.path.join(output_dir, spk, '*.npz'))

    mp_func = partial(mp_npz)
    with Pool(40) as p:
        rets = list(tqdm(p.imap(mp_func, npz_train), total=len(npz_train)))
        rets = list(filter(None, rets))
        rets.sort()
    
    os.makedirs('./filelists', exist_ok=True)
    with open('./filelists/filelist_vctk_train.txt', 'w') as f:
        for ret in rets:
            f.write('{}|{}\n'.format(ret[0], ret[1]))

# Example usage
input_dir = '/data1/DB/VCTK/VCTK/data/VCTK/wav48_silence_trimmed'
output_dir='/data1/projects/frepainter/preprocessed_data/VCTK'
make_filelist_vctk_train(input_dir, output_dir)


100%|██████████| 41289/41289 [00:01<00:00, 22255.27it/s]
